In [1]:
import cv2
from PIL import Image
import glob, os
import json

In [2]:
def norm2abs(bbx_coords, image_shape):
    x, y, w, h, = bbx_coords

    # get x and y
    x_reg = int(x * image_shape[1])
    y_reg = int(y * image_shape[0])
    w_reg = int(w * image_shape[1])
    h_reg = int(h * image_shape[0])

    bbx_coords_reg = [x_reg, y_reg, w_reg, h_reg]
    return bbx_coords_reg

In [3]:
# Check image dimensions, bounding box coordinates and labels
# We will compare the normalised and absolute coordinates, we assume the two json files are keyed identically
root = 'C:/Users/drago/Documents/gitrepos/recycle_robot/Autogathered_Dataset/1/'
directory = root + 'data/'
check_shape = False
# normalised coordinates
norm_labels_dir = root + 'Normalised_Labels_Dir_Adjusted.json'
abs_labels_dir = root + 'Completed_Labels_Dir_Adjusted.json'

# Open the json file containing the absolute labels
with open(abs_labels_dir) as abs_labels_json:
    abs_labels_dict = json.load(abs_labels_json)

# Open the json file containing the normalised labels
with open(norm_labels_dir) as norm_labels_json:
    norm_labels_dict = json.load(norm_labels_json)

#last_annotation_index = 0
# Iterate through each image in abs_labels, extracting filename and id
for file in [{'name': image['file_name'].split('/')[1], 'id': image['id']} for image in abs_labels_dict['images'][:]]:
    if file['name'].endswith('.png'):
        # Read in the image
        dataset_img = cv2.imread(directory + file['name'])
        
        # Extract the bounding boxes for each screw in the image
        label_ids = []
        #match_index = 0
        zero_bbx_area = False
        for index, abs_label in enumerate(abs_labels_dict['annotations'][:]):
            # Find the annotations that match the image
            # I want to know which images have annotations of area zero
            if abs_label['image_id'] == file['id']:
        
                ## Commented out because this new data wasn't downloaded as both normalised and absolute
                # # Read in normalised bounding box coordinates and convert to abs
                # # Here we assume the index is the same for both json files
                # norm_coords = norm2abs(norm_labels_dict['annotations'][index]['bbox'], (dataset_img.shape))

                # Read in absolute bounding box coordinates
                coordinates = abs_label['bbox']

                #Display image with bounding box
                start_point = (int(coordinates[0]), int(coordinates[1]))
                end_point = (int(coordinates[0] + coordinates[2]), int(coordinates[1] + coordinates[3]))
                
                print(start_point, end_point)
                # print((norm_coords[0], norm_coords[1]), (norm_coords[0] + norm_coords[2], norm_coords[1] + norm_coords[3]))
                print('\n')

                if abs_label['category_id'] == 2:
                    colour = (0, 0, 255)
                if abs_label['category_id'] == 4:
                    colour = (255, 0, 0)
                # draw the rectangle
                cv2.rectangle(dataset_img, start_point, end_point, color=colour, thickness= 1, lineType=cv2.LINE_8)

                if abs_label['area'] == 0:
                    zero_bbx_area = True
                    print("Image details: {}".format(file))
                    print("Label details: {}".format(abs_label))
                
                #match_index = index

        # Update the last annotation index
        #last_annotation_index = match_index

        if zero_bbx_area:
            print("Zero area bounding box found in image {}".format(file['name']))
            # display the output
            print("Dimensions {}: {}".format(file['name'], dataset_img.shape))
            cv2.imshow('imageRectangle', dataset_img)
            cv2.waitKey(0)
        else:
            continue
    else:
        continue

cv2.destroyAllWindows()

# # Check Github dataset iamge dimensions
# directory = 'C:/Users/drago/Documents/gitrepos/Cross-Recessed-Screw_Deep-Learning-Datasets/Training_Images/'
# for filename in os.listdir(directory):
#     if filename.endswith('.jpg'):
#         dataset_img = cv2.imread(directory + filename)
        
#         print("Dimensions {}: {}", (filename, dataset_img.shape))
#         continue
#     else:
#         continue

# directory = 'C:/Users/drago/Documents/gitrepos/Cross-Recessed-Screw_Deep-Learning-Datasets/Training_Images/'

Image details: {'name': '1167.png', 'id': 1}
Label details: {'id': 1, 'category_id': 2, 'image_id': 1, 'area': 483.0, 'bbox': [369.0, 365.0, 21.0, 23.0]}
(369, 365) (390, 388)


Image details: {'name': '1167.png', 'id': 1}
Label details: {'id': 2, 'category_id': 2, 'image_id': 1, 'area': 342.0, 'bbox': [279.0, 11.0, 19.0, 18.0]}
(279, 11) (298, 29)


Image details: {'name': '1167.png', 'id': 1}
Label details: {'id': 3, 'category_id': 4, 'image_id': 1, 'area': 528.0, 'bbox': [276.0, 9.0, 24.0, 22.0]}
(276, 9) (300, 31)


Image details: {'name': '1167.png', 'id': 1}
Label details: {'id': 4, 'category_id': 4, 'image_id': 1, 'area': 1056.0, 'bbox': [367.0, 360.0, 33.0, 32.0]}
(367, 360) (400, 392)


Image details: {'name': '298.png', 'id': 2}
Label details: {'id': 5, 'category_id': 2, 'image_id': 2, 'area': 676.0, 'bbox': [312.0, 584.0, 26.0, 26.0]}
(312, 584) (338, 610)


Image details: {'name': '298.png', 'id': 2}
Label details: {'id': 6, 'category_id': 4, 'image_id': 2, 'area': 1521.0, 

In [ ]:
# Convert git dataset to 480x640 dimensions
size = 480, 640

for infile in glob.glob(directory + "*.jpg"):
    file, ext = os.path.splitext(infile)
    file_name = os.path.basename(file)
    with Image.open(infile) as im:
        im.thumbnail(size)
        im.save(directory + 'thumbnail/' + file_name + '.jpg', "JPEG")

In [ ]:

# Check those dimensions match all images within their respective dataset
# NOTE: There are some rotated 90 deg
# Now, think of some creative way to handle the difference between the image sizes,
#  the new cameras image size and the size of model needed to take these as inputs?
# Do I need to scale down the size of the images for prediction? How would I make use of the predictions?
#  Would scaling them actually work?